# Data wrangling for "world_bank_project" Json file

data source: http://jsonstudio.com/resources/

# Import for Pyhton, Pandas and Json

In [1]:
#import pandas and json
import pandas as pd
import json

# Reading Json file

In [2]:
#loading json file and printing the first two rows
df = pd.read_json('world_bank_projects.json')

# Data Wrangling

# Find the 10 countries with most projects

In [3]:
# counting the number of projects for each country
country_count = df['countryname'].value_counts()

# 10 countries with most projects
country_count.head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

# Create a dataframe with the missing names filled in

In [4]:
# finding the data structure of project theme variable 'mjtheme_namecode' in the file
project_themes = df[['mjtheme_namecode']] 
project_themes.head(5)

,mjtheme_namecode
0,"[{'code': '8', 'name': 'Human development'}, {..."
1,"[{'code': '1', 'name': 'Economic management'},..."
2,"[{'code': '5', 'name': 'Trade and integration'..."
3,"[{'code': '7', 'name': 'Social dev/gender/incl..."
4,"[{'code': '5', 'name': 'Trade and integration'..."


In [5]:
# loops to parse project_themes into nested lists
for index, row in project_themes.iterrows():
    L = list()
    for i in range(index+1):
        a = [list(dict.values()) for dict in list(project_themes.loc[i].items())[0][1] ] #returning dict value into list
        L.append(a)

In [6]:
L[0]     #checking the data structure of L: [[[], []...]]

[['8', 'Human development'], ['11', '']]

In [7]:
from itertools import chain
lists = list(chain.from_iterable(L)) #exacting nested lists to be "[[], []...]"
df1 = pd.DataFrame(lists, columns=['code', 'name']) # changing the lists to Pandas dataframe

In [8]:
#filling blank with NaN value
import numpy as np
df1 = df1.replace('', np.nan, regex=True)
df1.head()

,code,name
0,8,Human development
1,11,NaN
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [9]:
# creating 'code' 'name' pairs
df2 = df1.dropna(how='any')
df3 = df2.drop_duplicates() 

In [10]:
new = pd.merge(left=df1, right=df3, on='code') #filling NaN valure with 'code' 'name' pairs 

In [11]:
#checing if NaN valures have been filled 
new.info()
new.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 1498
Data columns (total 3 columns):
code      1499 non-null object
name_x    1377 non-null object
name_y    1499 non-null object
dtypes: object(3)
memory usage: 46.8+ KB


,code,name_x,name_y
0,8,Human development,Human development
1,8,NaN,Human development
2,8,Human development,Human development
3,8,Human development,Human development
4,8,Human development,Human development


In [12]:
new = new[['name_y','code']]

# Find the top 10 major project themes (using column 'mjtheme_namecode')

In [13]:
new1 = new.groupby('name_y', as_index=False).count()
new1 = new1.sort_values('code', ascending=False)

In [14]:
new2 = new1.rename(columns={'name_y': 'name','code':'count'}).reset_index(drop=True)

In [15]:
new2.head(10)

,name,count
0,Environment and natural resources management,250
1,Rural development,216
2,Human development,210
3,Public sector governance,199
4,Social protection and risk management,168
5,Financial and private sector development,146
6,Social dev/gender/inclusion,130
7,Trade and integration,77
8,Urban development,50
9,Economic management,38


In [16]:
new1.code.sum()

1499